In [22]:
import pandas as pd 
import numpy as np 
import csv
from textblob import TextBlob
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from sklearn import preprocessing
import string
import re

In [ ]:
# df_sentimen = pd.read_excel("hasil klasifikasi.xlsx")
df_sentimen = pd.read_csv("xlhomedatatahun2018_2022.csv")

In [ ]:
df_sentimen['Text']= df_sentimen['Text'].str.lower()
df = df_sentimen

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, str(input_txt))
    for i in r:
        input_txt = re.sub(i, '', str(input_txt))
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df_sentimen['Text'], "@[\w]*")
# def remove(tweet):
    # #remove angka
    # tweet = re.sub('[0-9]+', '', tweet)
    
    # # remove stock market tickers like $GE
    # tweet = re.sub(r'\$\w*', '', tweet)
 
    # # remove old style retweet text "RT"
    # tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # # remove hashtags
    # # only removing the hash # sign from the word
    # tweet = re.sub(r'#', '', tweet)
    # return tweet
# df['remove_http'] = df['remove_user'].apply(lambda x: remove(x))
# df.drop_duplicates(subset ="remove_http", keep = 'first', inplace = True)
# df

In [ ]:
#import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
# stopwords_eng = stopwords.words('english')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)

    #emojis
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    tweet =  emoji_pattern.sub(r'', tweet)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (
            word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean

In [ ]:
df['tweet_clean'] = df['remove_user'].apply(lambda x: clean_tweets(x))
df.head(10)

In [ ]:
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text

df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))

df

In [ ]:
df.drop(df.columns[[0,1,2,3,4]], axis = 1, inplace = True)
df.drop_duplicates(subset ="Tweet", keep = 'first', inplace = True)

In [ ]:
# translate To english
# !pip install googletrans==3.1.0a0
import googletrans
from googletrans import Translator
from googletrans import LANGUAGES
import pandas as pd

# headers = ['Tweet', 'translation']
# data = pd.read_csv('./target.csv')
data = df
translator = Translator()

def trans(s):
  from googletrans import Translator
  translator = Translator()
  return (translator.translate(s).text)
data = data['Tweet'].apply(trans)

In [ ]:
data = data['Tweet'].apply(trans)

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:

data_tweet = list(data['tweet_clean'])
polaritas = 0

status = []
total_positif = total_negatif = total_netral = total = 0

for i, tweet in enumerate(data_tweet):
    analysis = TextBlob(tweet)
    polaritas += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif += 1
        status.append('Positif')
    elif analysis.sentiment.polarity == 0.0:
        total_netral += 1
        status.append('Netral')
    else:
        total_negatif += 1
        status.append('Negatif')

    total += 1 

In [ ]:
print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'\nTotal Data : {total}')


In [ ]:
status = pd.DataFrame({'klasifikasi': status})

In [ ]:
data['klasifikasi'] = status
data.tail()

In [ ]:
import matplotlib.pyplot as plt
def show_pie(label, data, legend_title) :
    fig, ax = plt.subplots(figsize=(8, 10), subplot_kw=dict(aspect='equal'))

    labels = [x.split()[-1] for x in label]

    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}% ({:d})".format(pct, absolute)

    wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data), 
                                      textprops=dict(color="w"))

    ax.legend(wedges, labels,
              title= legend_title,
              loc="center left",
              bbox_to_anchor=(1, 0, 0.5, 1))

    plt.setp(autotexts, size=10, weight="bold")
    plt.show()

label = ['Positif', 'Negatif', 'Netral']
count_data = [total_positif+1, total_negatif+1, total_netral]

show_pie(label, count_data, "Status")

In [ ]:
data

In [ ]:
data.drop(data.columns[[0]], axis = 1, inplace = True)
data.head(10)

In [ ]:
data.to_csv('TrainXl.csv')

# NAIVE BAYES

In [ ]:
df_sentimen = df

In [ ]:
Label = []
for index, row in df_sentimen.iterrows():
    if row["Label"] == 'positif':
        Label.append(1)
    else:
        Label.append(0)

df_sentimen["label"] = Label

In [ ]:
df_sentimen

In [ ]:
# s_1 = df_sentimen[df_sentimen['label']==0].sample(200,replace=True, random_state=0)
# s_2 = df_sentimen[df_sentimen['label']==1].sample(200,replace=True, random_state=0)
# df_pjj = pd.concat([s_1, s_2])

In [ ]:
# df_pjj

In [ ]:

x = df_sentimen['tweet']
y = df_sentimen['label']
print(x)
print (y.value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state = 7)

In [ ]:
print(x_train.shape)

In [ ]:
print(x_test.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#tfidf
vectorizer = TfidfVectorizer() 
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [ ]:
 print(x_train)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
print("\n================= confusion matrix ===================== ")
print(confusion_matrix(y_test, y_pred))
print("\n=========================================== ")
# accuracy score
print("accuracy score hasil prediksi adalah: ")
print(accuracy_score(y_test, y_pred) * 100)

# precision score
print("precision score hasil prediksi adalah: ")
print(precision_score(y_test, y_pred))

# recall score
print("recall score hasil prediksi adalah: ")
print(recall_score(y_test, y_pred))

In [ ]:
#tes uji data baru
tweet_pjj1 = ["pjj buat males belajar"]
tweet1 = tweet_pjj1
tweet_pjj2 = ["alhamdulillah nilai dapat bagus semangat pjjnya"]
tweet2 = tweet_pjj2
#merubah ke bentuk tfidf
tweet_pjj1 = vectorizer.transform(tweet_pjj1)
tweet_pjj2 = vectorizer.transform(tweet_pjj2)

In [ ]:
prediksi1 =classifier.predict(tweet_pjj1)
prediksi2 =classifier.predict(tweet_pjj2)

In [ ]:
if prediksi1 == 0:
    print(tweet1, "negatif")
if prediksi1 == 1:
    print(tweet1, "Positif")   
if prediksi2 == 0:
    print(tweet2, "negatif")
if prediksi2 == 1:
    print(tweet2, "Positif")


In [ ]:
df_databaru = pd.read_excel('Dataset/databarubersih.xlsx')

In [ ]:
df_databaru

In [ ]:
databaru = df_databaru['tweet']

In [ ]:
tweettabel = databaru

In [ ]:

#merubah ke bentuk number agar bisa di baca algoritma atau ML
databaru = vectorizer.transform(databaru)

In [ ]:
prediksidatabaru =classifier.predict(databaru)

In [ ]:
print(databaru)

In [ ]:
# print(tweettabel, *prediksidatabaru, sep = "\n") 
df_prediksidata = pd.DataFrame(tweettabel) 
df_databaru['polarity'] = prediksidatabaru

In [ ]:
df_databaru

In [ ]:
#df_databaru.to_excel('Dataset/hasilprediksi.xlsx',encoding='utf8', index=False)

# END CODE

# coDE

In [ ]:
# #from sklearn.model_selection import cross_val_score

# # x_test = vectorizer.transform(x_test)
# # y_pred = classifier.predict(x_test)
# cm = confusion_matrix(y_test, y_pred)
# print("\n================= confusion matrix ===================== ")
# print(cm)
# print('================akurasi Naive bayes 20% data tes=====================\n')
# print(accuracy_score(y_test, y_pred))
# # print('================Cross Validation=========================')
# # scores = cross_val_score(classifier, x_train, y_train, cv=5)
# # # Print the accuracy of each fold:
# # print(scores)

# # # Print the mean accuracy of all 5 folds
# # print(scores.mean())

In [ ]:
# from sklearn.model_selection import cross_val_predict
# y_crosspredict = cross_val_predict(classifier, x_train, y_train, cv=5)

# from sklearn.metrics import confusion_matrix
# conf_mat = confusion_matrix(y_train, y_crosspredict)
# print(conf_mat)

# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.plot(scores, 'o-')
# plt.xlabel('askurasi multinomial naive bayes')
# plt.ylabel('Cross-validated accuracy')